# 15.6 针对序列级和词元级应用程序微调BERT
- **目录**
  - 15.6.1 单文本分类
  - 15.6.2 文本对分类或回归
  - 15.6.3 文本标注
  - 15.6.4 问答

- 本章前几节中为自然语言处理应用设计了不同的模型。
  - 例如基于循环神经网络、卷积神经网络、注意力和多层感知机。
- 这些模型在有空间或时间限制的情况下是有帮助的，
  - 但是，为每个自然语言处理任务精心设计一个特定的模型实际上是不可行的。
- 在 14.8节中，我们介绍了一个名为BERT的预训练模型，该模型可以对广泛的自然语言处理任务进行最少的架构更改。
  - 一方面，在提出时，BERT改进了各种自然语言处理任务的技术水平。
  - 另一方面，正如在 14.10节中指出的那样，原始BERT模型的两个版本分别带有1.1亿和3.4亿个参数。
- 因此，当有足够的计算资源时，我们可以考虑为下游自然语言处理应用微调BERT。
- 下面，我们将**自然语言处理应用的子集概括为序列级和词元级** 。
  * 在序列层次上，介绍了在单文本分类任务和文本对分类（或回归）任务中，如何将文本输入的BERT表示转换为输出标签。
  * 在词元级别，我们将简要介绍新的应用，如文本标注和问答，并说明BERT如何表示它们的输入并转换为输出标签。
  * 在微调期间，不同应用之间的BERT所需的“**最小架构更改**”是额外的全连接层。
  * 在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的。
 

- **要点：**
  - 设计多种NLP模型存在实际挑战。
  - BERT是一个适用于多个NLP任务的预训练模型。
  - 对于序列级和词元级任务，BERT只需最小架构更改。
  - BERT模型参数量大，需要充足的计算资源。
  - 微调BERT时，除新增层参数外，所有预训练参数都需微调。

---------------
- **说明：**
- **（1）何为"序列级"和"词元级"任务？**
  - "序列级"和"词元级"任务是自然语言处理中的两类基本任务，它们按照处理的粒度和任务的性质进行分类：
    - 序列级任务关注整个文本序列的意义和分类，而词元级任务则关注对文本中具体词元的识别和分类。
    - 在应用BERT或其他预训练模型进行微调时，这种区分帮助确定需要进行的架构调整，以及对应的损失函数和输出层设计。
  - **序列级任务**（Sequence-level tasks）关注的是整个文本序列的处理和理解。在这类任务中，模型的输出通常是关于整个输入文本的单一结果。这些任务的特点是结果不需要对每个单词或词元进行区别，而是得到一个整体的输出。序列级任务的例子包括：
    - **文本分类(Text Classification)**：判断整个文本的类别，如垃圾邮件检测、情感分析。
    - **自然语言推断(Natural Language Inference, NLI)**：给定一个前提和一个假设，判断假设是真实的，假的还是无法确定的。
    - **情感分析(Sentiment Analysis)**：判断文本所表达情感的倾向，例如正面、负面或中性。
  - **词元级任务**（Token-level tasks）则涉及对文本中的单个词元（如单词或字符）进行处理和理解。模型必须对文本中的每一个词元给出具体的输出或判断。词元级任务的例子包括：
    - **命名实体识别(Named Entity Recognition, NER)**：识别文本中的特定命名实体（如人名、地点、机构名称）。
    - **词性标注(Part-of-Speech Tagging, POS)**：为文本中的每个词分配适当的词性（如名词、动词、形容词等）。
    - **语义角色标注(Semantic Role Labeling, SRL)**：分析句子中各个词汇的语义角色，如谁是行动的执行者、目标对象等。
    - **问答(Question Answering, QA)**：在问答任务中识别回答问题所需的关键词或短语。

- **（2）如何理解：在微调期间，不同应用之间的BERT所需的“最小架构更改”是额外的全连接层？**
  - 当使用BERT模型来微调不同的下游自然语言处理任务时，通常不需要对BERT的核心架构进行大幅度的修改。相反，只需要增加一个额外的全连接层（即一个线性层或密集层），以适应特定任务的输出。
  - “最小架构更改”意味着：
    - **保持BERT核心**: 预训练的BERT模型核心，包括它的层和参数，是不变的。这些层已经通过大规模的数据集学习了语言的通用特征和表示。
    - **额外全连接层的角色**:
       - 对于**分类任务**，这个额外的全连接层通常会输出一个固定大小的向量，其维度与分类任务的类别数量相匹配。
       - 对于**回归任务**，这个层通常输出一个单一的数值，表示某个连续的目标变量，比如两个句子间的语义相似度。
    - **任务特异性训练**: 在微调过程中，虽然BERT的参数也会进行微调来更好地适应特定任务，但新增的全连接层参数是特定于任务的，会从零开始训练。这表明预训练BERT模型已经具备了很强的语言表示能力，通常只需少量的修改即可适用于不同的NLP任务。

- **（3）如何理解：在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的？**
  - 这句话说明了在BERT模型用于具体NLP任务时，针对任务特定输出所添加的新层是完全重新学习的，而预训练模型中的参数则是基于它们之前学习到的知识进行细微调整的。
  - 这允许模型既能够利用预训练中获得的丰富语言知识，又能够适应特定任务的特殊需求。
  - 同时说明了在使用预训练的BERT模型对特定下游任务进行微调时的两个关键步骤：
     -  **额外层的参数学习**:
        - 当BERT模型被应用于一个具体的下游任务时，需要一个额外层（通常是一个或多个全连接层）来适配特定的任务需求，如分类标签的预测。
        - 这个额外层是为特定任务新加的，其参数在开始微调前没有被训练过，因此是从零开始学习的。
        - 也就是说，这些参数是随机初始化的，并会在下游任务的监督学习过程中训练和调整。
     - **预训练BERT模型参数的微调**:
       - 预训练的BERT模型已经通过大量的通用数据进行了训练，这使它学到了语言的各种特征和规律。
       - 在下游任务中，这些预训练的参数不会重新开始学习，而是从预训练时已经学到的知识状态开始，通过进一步的训练（微调）来适应具体任务。
       - 微调允许模型在保留预训练时学到的通用知识的同时，根据特定任务的数据进行调整，以更好地执行特定的下游任务。

- **（4）在微调过程中，BERT模型的参数是如何更新和调整的？**
  - 在微调过程中，BERT模型的参数更新和调整，遵循标准的深度学习训练过程，特别是使用梯度下降法。以下是使用微调BERT模型时的参数更新步骤：
    - 1. **初始化**:
       - 在开始微调之前，BERT模型的参数会被初始化为预训练阶段学到的权重。
    - 2. **数据准备**:
      - 微调所用的数据集需要根据具体任务进行预处理，如分类标签或者其他相关的任务特定信息。
    - 3. **添加额外层**:
      - 通常情况下，会在BERT模型的顶部增加一个或者几个全连接层（对于不同的任务可有所不同）以适应特定的输出要求。
    - 4. **前向传播**:
      - 在训练阶段，输入数据通过BERT模型进行前向传播，经过多层的变换最终到达额外添加的全连接层，得到输出结果。
    - 5. **计算损失**:
      - 根据任务的具体需求（例如分类、回归、序列标注等），使用一个损失函数来计算模型当前输出和真实标签之间的差距。
    - 6. **反向传播**:
      - 损失函数的结果用来进行反向传播。在这个过程中，通过链式法则计算模型每层参数相对于损失的梯度。
    - 7. **参数更新**:
      - 使用梯度和学习率来更新模型参数。通常使用优化算法如Adam，这种算法可以通过调整学习速率来改进梯度下降。
    - 8. **重复迭代**:
      - 将整个数据集分批次输入模型，并重复上述的训练步骤，每个批次更新一次模型的参数。
    - 9. **学习率调整**:
      - 有时在训练过程中会根据一定的策略调整学习率，如逐步衰减或使用预热阶段的学习率调整策略。
    - 10. **早停**:
      - 为了防止过拟合，通常会在验证集上跟踪模型的性能，如果性能在一定迭代后没有显著提升可能会提前结束训练。
  - 这个微调过程涉及所有参数的细微调整，以便学习从预训练阶段转移到特定任务的完成情况。
  - 最终随着训练的进行，模型逐渐从通用的语言理解调整到特定任务的需求。

--------------

## 15.6.1 单文本分类
- **单文本分类**将单个文本序列作为输入，并输出其分类结果。
- 除了我们在这一章中探讨的情感分析之外，**语言可接受性语料库（Corpus of Linguistic Acceptability，COLA）** 也是一个单文本分类的数据集，它的要求判断给定的句子在语法上是否可以接受 。
- 例如，“I should study.”是可以接受的，但是“I should studying.”不是可以接受的。
<center><img src='../img/bert-one-seq.svg'></center>
<center>图15.6.1 微调BERT用于单文本分类应用，如情感分析和测试语言可接受性（这里假设输入的单个文本有六个词元）</center><br>
- 14.8节描述了BERT的输入表示。
  - BERT输入序列明确地表示单个文本和文本对。
  - 其中特殊分类标记“&lt;cls&gt;”用于序列分类，而特殊分类标记“&lt;sep&gt;”标记单个文本的结束或分隔成对文本。
- 如图15.6.1所示，在单文本分类应用中，特殊分类标记“&lt;cls&gt;”的BERT表示对整个输入文本序列的信息进行编码。
- 作为输入单个文本的表示，它将被送入到由全连接（稠密）层组成的小多层感知机中，以输出所有离散标签值的分布。

- **要点：**
  -  **单文本分类**: 输入是单个文本序列，输出是这段文本的分类。
  - **数据集举例**: 情感分析和语言可接受性语料库（COLA）检测句子的语法是否可以接受。
  - **句子示例**: "I should study."（可以接受）与"I should studying."（不可接受）。
  - **BERT输入**: 使用特殊标记"\<cls\>"表示序列的开始，用于分类任务；用"\<sep\>"标记文本结尾或分隔文本对。
  - **模型结构**: 在BERT的"\<cls\>"标记表示编码了整个文本序列的信息，这个信息随后被使用在一个全连接层上，来预测不同标签值的分布。

## 15.6.2 文本对分类或回归
- 在本章中，我们还研究了自然语言推断。它属于**文本对分类**，这是一种对文本进行分类的应用类型。
- 以一对文本作为输入但输出连续值，**语义文本相似度**是一个流行的“文本对回归”任务。
- 这项任务评估句子的语义相似度。
  - 例如，在语义文本相似度基准数据集（Semantic Textual Similarity Benchmark）中，句子对的相似度得分是从0（无语义重叠）到5（语义等价）的分数区间。
- 我们的目标是预测这些分数。来自语义文本相似性基准数据集的样本包括（句子1，句子2，相似性得分）：
  * "A plane is taking off."（“一架飞机正在起飞。”），"An air plane is taking off."（“一架飞机正在起飞。”），5.000分;
  * "A woman is eating something."（“一个女人在吃东西。”），"A woman is eating meat."（“一个女人在吃肉。”），3.000分;
  * "A woman is dancing."（一个女人在跳舞。），"A man is talking."（“一个人在说话。”），0.000分。
<center><img src='../img/bert-two-seqs.svg'></center>
<center>图15.6.2 文本对分类或回归应用的BERT微调，如自然语言推断和语义文本相似性（假设输入文本对分别有两个词元和三个词元）</center><br>
- 与图15.6.1中的单文本分类相比，图15.6.2中的文本对分类的BERT微调在输入表示上有所不同。
- 对于文本对回归任务（如语义文本相似性），可以应用细微的更改，例如输出连续的标签值和使用均方损失：它们在回归中很常见。

- **要点：**
  - **文本对分类**: 以两段文本作为输入，分类任务的输出是它们之间**关系的类别**。
  - **文本对回归**: 输入为文本对，任务输出一个连续值，表示文本间的**语义相似度**。
  - **语义文本相似度**: 一种文本对回归任务，用分数（0-5）来评估两个句子的语义相似性。
  - **模型结构**: BERT模型微调用于处理输入的文本对，"\<cls\>"标记用于编码两个文本合并后的整体信息。
  - **损失函数**: 文本对回归任务可以使用均方损失来预测连续的相似度分数。

## 15.6.3 文本标注
- 现在让我们考虑词元级任务，比如**文本标注（text tagging）**，其中每个词元都被分配了一个标签。
- 在文本标注任务中，**词性标注**为每个单词分配词性标记（例如，形容词和限定词）。
- 根据单词在句子中的作用。
  - 如在Penn树库II标注集中，句子“John Smith‘s car is new”应该被标记为“NNP（名词，专有单数）NNP POS（所有格结尾）NN（名词，单数或质量）VB（动词，基本形式）JJ（形容词）”。
<center><img src='../img/bert-tagging.svg'></center>
<center>图15.6.3 文本标记应用的BERT微调，如词性标记。假设输入的单个文本有六个词元。</center>
- 图15.6.3中说明了文本标记应用的BERT微调。
- 与图15.6.1相比，唯一的区别在于，在文本标注中，输入文本的**每个词元**的BERT表示被送到相同的额外全连接层中，以输出词元的标签，例如**词性标签**。

- **要点：**
  - 文本标记是一种词元级任务，每个词元被分配一个标签。
  - 词性标注是文本标记的一种形式，为每个单词分配词性（如形容词、限定词）。
  - 文本标记根据单词在句子中的语法作用进行。
  - 例如，根据Penn树库II标注集，"John Smith's car is new" 标记为 "NNP NNP POS NN VB JJ"。
  - 文本标记可以通过对BERT模型进行微调来实现，每个词元的表示通过额外的全连接层，以获取其标签。

## 15.6.4 问答
- 作为另一个词元级应用，**问答**反映阅读理解能力。
- 例如，斯坦福问答数据集（Stanford Question Answering Dataset，SQuAD v1.1）**由阅读段落和问题组成，其中每个问题的答案只是段落中的一段文本（文本片段）**  。
- 举个例子,考虑一段话：
  - “Some experts report that a mask's efficacy is inconclusive.However,mask makers insist that their products,such as N95 respirator masks,can guard against the virus.”（“一些专家报告说面罩的功效是不确定的。然而，口罩制造商坚持他们的产品，如N95口罩，可以预防病毒。”）还有一个问题“Who say that N95 respirator masks can guard against the virus?”（“谁说N95口罩可以预防病毒？”）。
  - 答案应该是文章中的文本片段“mask makers”（“口罩制造商”）。
- 因此，**SQuAD v1.1的目标是在给定问题和段落的情况下预测段落中文本片段的开始和结束。**
<center><img src='../img/bert-qa.svg'></center>
<center>图15.6.4 对问答进行BERT微调（假设输入文本对分别有两个和三个词元）</center><br>
- 为了微调BERT进行问答，在BERT的输入中，将问题和段落分别作为第一个和第二个文本序列。
- 为了预测文本片段开始的位置，相同的额外的全连接层将把来自位置$i$的任何词元的BERT表示转换成标量分数$s_i$。
- 文章中所有词元的分数还通过softmax转换成概率分布，从而为文章中的每个词元位置$i$分配作为文本片段开始的概率$p_i$。
- 预测文本片段的结束与上面相同，只是其额外的全连接层中的参数与用于预测开始位置的参数无关。
- 当预测结束时，位置$i$的词元由相同的全连接层变换成标量分数$e_i$。 
- 图15.6.4描述了用于问答的微调BERT。
- 对于问答，监督学习的训练目标就像最大化真实值的开始和结束位置的对数似然一样简单。
  - 当预测片段时，我们可以计算从位置$i$到位置$j$的有效片段的分数$s_i + e_j$（$i \leq j$），并输出分数最高的跨度。

- **要点：**
  -  问答任务是反映阅读理解的词元级应用。
  - SQuAD v1.1数据集包括阅读段落和问题，答案是段落中的一个文本片段。
  - 目标是预测给定问题的答案在段落中的开始和结束位置。
  - 微调BERT时，问题和段落分别作为输入文本的两个序列。
  - 一个全连接层把BERT的每个词元表示转换成开始位置的分数，这些分数经softmax转换成概率分布。
  - 另一个独立的全连接层用于转换结束位置的分数，并进行相同的概率分布转换。
  - 训练包括最大化实际答案开始和结束位置的对数似然。
  - 预测答案文本片段通过计算和指定开始和结束位置的分数组合，并选择分数最高的跨度。

## 小结

* 对于序列级和词元级自然语言处理应用，BERT只需要最小的架构改变（额外的全连接层），如单个文本分类（例如，情感分析和测试语言可接受性）、文本对分类或回归（例如，自然语言推断和语义文本相似性）、文本标记（例如，词性标记）和问答。
* 在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的。